In [13]:
# Import libraries

import warnings
from resources import *
import datetime
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras import metrics, losses, optimizers


warnings.filterwarnings("ignore")

In [14]:
# % run resources.py

In [15]:
# Load user items data
recdata = pd.read_csv('recdata.csv', index_col=0)
recdata = recdata.rename(columns={'variable': 'id', 'value': 'owned'})
recdata.head()

,uid,id,owned
0,0,10,1.0
1,1,10,1.0
2,3,10,1.0
3,4,10,1.0
4,10,10,1.0


## Preprocessing

In [16]:
# Use create_interaction_matrix function
interactions = create_interaction_matrix(
    df=recdata, user_col="uid", item_col="id", rating_col="owned"
)


In [17]:
# Create user dictionary using helper function
user_dict = create_user_dict(interactions=interactions)

# Load games data
gamesdata = pd.read_csv('gamesdata.csv', index_col=0)

# Create game dictionary using helper function
games_dict = create_item_dict(df=gamesdata, id_col='id', name_col='title')

In [18]:
# Load the data
data = interactions.T
# data['y'] = data.apply(lambda row: row.values, axis=1)
data.head()

uid,0,1,2,3,4,5,6,7,8,10,...,29986,29987,29988,29993,29994,29995,29996,29997,29998,29999
id,,,,,,,,,,,,,,,,,,,,,
10,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
20,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
30,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
40,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
50,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Train TF model

In [48]:
# Get recs functions

def get_game_title(game_id, game_data):
    title = None
    if game_id in game_data:
        title = game_data[game_id]
    return title



def get_recommendations(model, data, user_id, n, games_dict):
    # Get the index of the user in the data
    user_index = data.index.get_loc(user_id)
    # Get the user's interactions with the games
    interactions = data.iloc[user_index].values

    # Generate the input features for the model
    input_features = interactions.reshape(1, -1)

    predictions = model.predict(input_features)[0]

    # Get the indices of the top N recommendations
    top_indices = predictions.argsort()[-n:][::-1]

    # Get the titles of the top N recommendations
    game_titles = []
    for i in top_indices:
        game_id = data.columns[i]
        game_title = get_game_title(game_id, games_dict)
        game_titles.append(game_title)

    # Print the user's games and the top N recommendations
    print("User's games:")
    user_games = data.iloc[user_index][data.iloc[user_index] != None].index.tolist()

    for game_id in user_games:
        game_title = get_game_title(game_id, games_dict)
        print("- {}".format(game_title))

    print("Top {} recommendations:".format(n))
    for i, game_title in enumerate(game_titles):
        print("{}. {}".format(i + 1, game_title))


In [20]:

# model = Sequential([
#     layers.Dense(64, activation='relu', input_shape=(len(data.columns) - 1,)),
#     layers.Dense(32, activation='relu'),
#     layers.Dense(1, activation='sigmoid')
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'mse'])

# # Split the data into training and testing sets
# train_data = data.sample(frac=0.8, random_state=0)
# test_data = data.drop(train_data.index)

# # Define the input and output columns
# X_train = train_data.iloc[:, :-1].values
# y_train = np.stack(train_data.iloc[:, -1].values, axis=0)  # convert to 2D numpy array
# X_test = test_data.iloc[:, :-1].values
# y_test = np.stack(test_data.iloc[:, -1].values, axis=0)  # convert to 2D numpy array

# # Train the model
# model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=1, callbacks=[tensorboard_callback])

### Create model

In [21]:

# Log tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Dense(32, activation='relu', input_shape=(data.shape[1],)),
    Dense(16, activation='relu'),
    Dense(data.shape[1], activation='sigmoid')
])

metrics = [
            metrics.Precision(),
            metrics.Recall(),
            metrics.AUC(from_logits=True),
           ]
loss = losses.BinaryCrossentropy(from_logits=True),
optimizer = optimizers.Adam(learning_rate=0.01)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train the model on the training data
model.fit(train_data, train_data, epochs=30, batch_size=16, validation_data=(test_data, test_data), callbacks=[tensorboard_callback])

Epoch 1/30
433/433 [==============================] - 15s 33ms/step - loss: 0.0609 - precision_1: 0.1866 - recall_1: 0.0325 - auc_1: 0.8626 - val_loss: 0.0348 - val_precision_1: 0.6605 - val_recall_1: 0.0416 - val_auc_1: 0.9357
Epoch 2/30
433/433 [==============================] - 13s 31ms/step - loss: 0.0370 - precision_1: 0.6432 - recall_1: 0.0383 - auc_1: 0.9178 - val_loss: 0.0356 - val_precision_1: 0.6870 - val_recall_1: 0.0388 - val_auc_1: 0.9459
Epoch 3/30
433/433 [==============================] - 14s 31ms/step - loss: 0.0343 - precision_1: 0.6375 - recall_1: 0.0452 - auc_1: 0.9246 - val_loss: 0.0347 - val_precision_1: 0.7014 - val_recall_1: 0.0404 - val_auc_1: 0.9321
Epoch 4/30
433/433 [==============================] - 13s 31ms/step - loss: 0.0335 - precision_1: 0.6317 - recall_1: 0.0488 - auc_1: 0.9279 - val_loss: 0.0341 - val_precision_1: 0.6683 - val_recall_1: 0.0517 - val_auc_1: 0.9407
Epoch 5/30
433/433 [==============================] - 13s 31ms/step - loss: 0.0333 - pre

In [26]:
# Evaluate the model
test = model.evaluate(test_data, test_data, verbose=2)


55/55 - 2s - loss: 0.0334 - precision_1: 0.6491 - recall_1: 0.0474 - auc_1: 0.9471 - 2s/epoch - 39ms/step


In [49]:
get_recommendations(model, data, user_id=10, n=5, games_dict=games_dict)

1/1 [==============================] - 0s 20ms/step
User's games:
- None
- None
- None
- None
- None
- None
- None
- None
- None
- Counter-Strike
- None
- None
- None
- None
- None
- None
- None
- None
- Team Fortress Classic
- None
- None
- None
- None
- None
- None
- None
- None
- None
- Day of Defeat
- None
- None
- None
- None
- None
- None
- None
- Deathmatch Classic
- None
- None
- None
- None
- None
- None
- None
- None
- None
- Half-Life: Opposing Force
- None
- None
- None
- None
- None
- None
- None
- None
- Ricochet
- None
- None
- None
- None
- None
- None
- None
- None
- Half-Life
- None
- None
- None
- None
- None
- None
- None
- None
- Counter-Strike: Condition Zero
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- None
- 